In [27]:
import libpgm
from libpgm.lgbayesiannetwork import LGBayesianNetwork
from libpgm.graphskeleton import GraphSkeleton

In [28]:
num_sources = 6

bins = ["d" + str(i) for i in range(16)]
sources = ["source" + str(i) for i in range(num_sources)]
factors = ["humidity", "temp"]
nodes = bins + sources + factors

In [29]:
edges = []
for factor in factors:
    for source in sources:
        for b in bins:
            edges.append([source, b])
        edges.append([factor, source])

[['source0', 'd0'],
 ['source0', 'd1'],
 ['source0', 'd2'],
 ['source0', 'd3'],
 ['source0', 'd4'],
 ['source0', 'd5'],
 ['source0', 'd6'],
 ['source0', 'd7'],
 ['source0', 'd8'],
 ['source0', 'd9'],
 ['source0', 'd10'],
 ['source0', 'd11'],
 ['source0', 'd12'],
 ['source0', 'd13'],
 ['source0', 'd14'],
 ['source0', 'd15'],
 ['humidity', 'source0'],
 ['source1', 'd0'],
 ['source1', 'd1'],
 ['source1', 'd2'],
 ['source1', 'd3'],
 ['source1', 'd4'],
 ['source1', 'd5'],
 ['source1', 'd6'],
 ['source1', 'd7'],
 ['source1', 'd8'],
 ['source1', 'd9'],
 ['source1', 'd10'],
 ['source1', 'd11'],
 ['source1', 'd12'],
 ['source1', 'd13'],
 ['source1', 'd14'],
 ['source1', 'd15'],
 ['humidity', 'source1'],
 ['source2', 'd0'],
 ['source2', 'd1'],
 ['source2', 'd2'],
 ['source2', 'd3'],
 ['source2', 'd4'],
 ['source2', 'd5'],
 ['source2', 'd6'],
 ['source2', 'd7'],
 ['source2', 'd8'],
 ['source2', 'd9'],
 ['source2', 'd10'],
 ['source2', 'd11'],
 ['source2', 'd12'],
 ['source2', 'd13'],
 ['source2',

In [30]:
skeleton = OrderedSkeleton(nodes, edges) #load with txt file

TypeError: __init__() takes at most 2 arguments (3 given)

In [ ]:
model = LGBayesianNetwork(skeleton)